In [23]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import steplib as msl
import dataloader as dl
%matplotlib inline

In [24]:
all_files = dl.getOxData()

In [32]:
import numpy as np
end = 60000

def getSteps(file, window_size, threshold):
    file_data = pd.read_csv(file)

    expected = np.where(file_data["annotation"] == 1)[0]

    data1 = file_data["x"] ** 2 + file_data["y"] ** 2 + file_data["z"] ** 2
    accelerometer = data1.apply(lambda x: x ** 0.5)
    data1 = msl.apply_filter(accelerometer, dl.getFilterFor100hz())
    data1 = msl.maximum_difference(data1, window_size)
    data1 = msl.detect_outliers(data1, threshold)
    data1 = msl.post_processing(data1, 200, 100)
    calculated = np.where(data1 == 1)[0]
    return calculated, expected , accelerometer


In [26]:
def chart(data_in):
    (calculated, expected, accelerometer) = data_in
    x=range(len(accelerometer))
    y=accelerometer
    plt.figure(figsize=(10,5))
    plt.plot(x,y, color='red', marker='o')
    plt.xlabel('index')
    plt.ylabel('magnitude')
    #show x label every 800
    #add rectangles
    for e in expected:
        plt.axvline(x =e, color ='g', label ='axvline - full height', ymin=0.5, alpha=0.8)
    for c in calculated:
        plt.axvline(x =c, color ='b', label ='axvline - full height', ymax=0.5, alpha=0.8)


    plt.show()

In [27]:
def getGroundTruth(i):
    file = all_files[i]
    file_data = pd.read_csv(file)
    expected = np.where(file_data["annotation"] == 1)[0]
    return len(expected)

In [28]:
all_files[0]

'DataOx\\Hip_100Hz\\P01_hip100.csv'

In [29]:
#where name contains regular
#all_files = [file for file in all_files if "_Regular" in file]

In [31]:
from statistics import mean
from scipy.optimize import minimize

def oneObjective(x,i):
    result = getSteps(all_files[i], int(x[0]), x[1])
    return abs(len(result[1]) - len(result[0])) / len(result[1])

def objective(x):
    return mean([oneObjective(x, i) for i in range(2, 30)])*100

x0 = np.array([35, 1.2])
res = minimize(objective, x0, method='nelder-mead', options={'xatol': 1e-1, 'disp': True, 'maxiter': 5})
print(res.x)
print(res.fun)

[14.4375   1.88625]
196.07686011033454


C:\Users\trebu\AppData\Local\Temp\ipykernel_20816\813204043.py:14: RuntimeWarning: Maximum number of iterations has been exceeded.
  res = minimize(objective, x0, method='nelder-mead', options={'xatol': 1e-1, 'disp': True, 'maxiter': 5})


In [ ]:
def oneObjective2(x,i):
    result = getSteps(all_files[i], int(x[0]), x[1])
    return len(result[1]) - len(result[0])

In [ ]:

for i in range(0, 30):
    print(oneObjective2(res.x, i), getGroundTruth(i))

In [33]:
%matplotlib qt
chart(getSteps(all_files[2],int(res.x[0]), res.x[1]))

In [ ]:
%matplotlib qt
data = dl.getSheepData()
chart(getSteps(data[0],5, 4))

In [ ]:
import time

def getStepsWithTime(file, window_size=20, threshold=1.15):
    start_time = time.time()
    file_data = pd.read_csv("DataOx\\Hip_100Hz\\" + file)[0:end]
    print("Time taken for reading file: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    expected = np.where(file_data["annotation"] == 1)[0]
    print("Time taken for np.where: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    data1 = file_data["x"] ** 2 + file_data["y"] ** 2 + file_data["z"] ** 2
    accelerometer = data1.apply(lambda x: x ** 0.5)
    print("Time taken for calculating accelerometer: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    data1 = msl.apply_filter(accelerometer, [0.0167855014066363,	0.0470217363830679,	0.121859539593060,	0.198782647391950,	0.231101150450572,	0.198782647391950,	0.121859539593060,	0.0470217363830679,	0.0167855014066363])
    print("Time taken for apply_filter: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    data1 = msl.modified_pan_tompkins_scoring(data1, window_size)
    print("Time taken for modified_pan_tompkins_scoring: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    data1 = msl.detect_outliers(data1, threshold)
    print("Time taken for detect_outliers: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    data1 = msl.post_processing(data1, 250, 100)
    print("Time taken for post_processing: %s seconds" % (time.time() - start_time))

    start_time = time.time()
    calculated = np.where(data1 == 1)[0]
    print("Time taken for final np.where: %s seconds" % (time.time() - start_time))

    return calculated, expected , accelerometer
